***
# `COVID-19 Dashboard`
***
<!-- ***
# <font color='blue'>COVID-19 Dashboard</font>
*** -->

> Coronavirus disease (COVID-19) is an infectious disease caused by a newly discovered coronavirus. Most people infected with 
the COVID-19 virus will experience mild to moderate respiratory illness and recover without requiring special treatment. Older 
people, and those with underlying medical problems like cardiovascular disease, diabetes, chronic respiratory disease, and 
cancer are more likely to develop serious illness. The best way to prevent and slow down transmission is be well informed about 
the COVID-19 virus, the disease it causes and how it spreads. Protect yourself and others from infection by washing your hands 
or using an alcohol based rub frequently and not touching your face.

In [1]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.core.display import display, HTML
import ipywidgets as widgets
import pandas as pd
import numpy as np
import folium
import plotly.graph_objects as go

In [2]:
#HTML("<img src=https://www.ikg-wien.at/wp-content/uploads/2020/03/Coronavirus-2.jpg, width=650 >")
HTML("<img src=https://www.astho.org/uploadedImages/Programs/Infectious_Disease/Corona/COVID%20Banner.jpg?n=8180, width=950 >")

In [3]:
death_df = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv")
confirmed_df = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv")
recovered_df = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv")
country_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/web-data/data/cases_country.csv')

In [4]:
country_df.columns = map(str.lower, country_df.columns)
confirmed_df.columns = map(str.lower, confirmed_df.columns)
recovered_df.columns = map(str.lower, recovered_df.columns)
death_df.columns = map(str.lower, death_df.columns)

In [5]:
confirmed_df = confirmed_df.rename(columns = {'province/state':'state', 'country/region':'country'})
recovered_df = recovered_df.rename(columns = {'province/state':'state', 'country/region':'country'})
death_df = death_df.rename(columns = {'province/state':'state', 'country/region':'country'})
country_df = country_df.rename(columns = {'country_region':'country'})

In [6]:
sorted_country_df = country_df.sort_values('confirmed', ascending=False)

In [7]:
new_data = pd.read_csv("https://raw.githubusercontent.com/microsoft/Bing-COVID-19-Data/master/data/Bing-COVID19-Data.csv")

***
# `Corona virus spread over Map`
***

In [8]:
# map_data=new_data.groupby(['Latitude','Longitude'])
# world_map = folium.Map(location=[20.5,78.9], tiles='cartodbpositron', zoom_start=4, max_zoom= 6, min_zoom=2)

# for i, j in map_data:
#     j.fillna(0, inplace=True)
#     if j['AdminRegion2'].iloc[-1]==0:
#         if j['AdminRegion1'].iloc[-1]==0:
#             region=j['Country_Region'].iloc[-1]
#         else:
#             region=j['AdminRegion1'].iloc[-1]
#     else:
#         region=j['AdminRegion2'].iloc[-1]
#     folium.Circle(
#         location=[i[0], i[1]],
#         fill=True,
#         radius = (int((np.log((j['Confirmed'].iloc[-1])+1.00001)))+0.2)*6000,
#         fill_color = 'red',
#         color = 'tomato',
#         tooltip = "<div style='margin: 0; background-color: powderblue; color: blue;'>"+
#                     "<h4 style='text-align:center;font-weight: bold'>"+str(region) + "</h4>"
#                     "<hr style='margin:10px;color: white;'>"+
#                     "<ul style='color: red;;list-style-type:circle;align-item:left;padding-left:20px;padding-right:20px'>"+
#                         "<li>Confirmed: "+str((j['Confirmed'].iloc[-1]))+"</li>"+
#                         "<li>Deaths:   "+str((j['Deaths'].iloc[-1]))+"</li>"+
#                         "<li>Recovered: "+str((j['Recovered'].iloc[-1]))+ "</li>"+
#                     "</ul></div>",
#         ).add_to(world_map)

    
#world_map    

world_map = folium.Map(location=[20.5,78.9], tiles='openstreetmap', zoom_start=4, max_zoom= 6, min_zoom=2)

for i in range(len(confirmed_df)):
    folium.Circle(
        location=[confirmed_df.iloc[i]['lat'], confirmed_df.iloc[i]['long']],
        fill=True,
        radius = (int((np.log(confirmed_df.iloc[i,-1]+1.00001)))+0.2)*50000,
        fill_color = 'blue',
        color = 'red',
        tooltip = "<div style='margin: 0; background-color: powderblue; color: blue;'>"+
                    "<h4 style='text-align:center;font-weight: bold'>"+confirmed_df.iloc[i]['country'] + "</h4>"
                    "<hr style='margin:10px;color: white;'>"+
                    "<ul style='color: blue;;list-style-type:circle;align-item:left;padding-left:20px;padding-right:20px'>"+
                        "<li>Confirmed: "+str(confirmed_df.iloc[i,-1])+"</li>"+
                        "<li>Deaths:   "+str(death_df.iloc[i,-1])+"</li>"+
                        "<li>Death Rate: "+ str(np.round(death_df.iloc[i,-1]/(confirmed_df.iloc[i,-1]+1.00001)*100,2))+ "</li>"+
                    "</ul></div>",
        ).add_to(world_map)   
world_map    

NameError: name 'world_map' is not defined

***
# `Current cases`
***

# Select your Country to get data:

In [ ]:
def country_data(country):
    df_list = [confirmed_df, death_df, recovered_df]
    l=[]
    for df in df_list:       
        if country == 'World' or country == 'world':
                data = np.sum(np.asarray(df.iloc[:,4:]),axis = 0)
                l.append(data[-1])                
        else:
            data = np.sum(np.asarray(df[df['country'] == country].iloc[:,20:]),axis = 0)
            l.append(data[-1])
    l=["{:,}".format(x) for x in l]
    display(HTML("<div style = 'background-color: #504e4e; padding: 30px '>" +
             "<span style='color: yellow; font-size:30px;'> Confirmed: "  + str(l[0]) +"</span>" +
             "<span style='color: red; font-size:30px;margin-left:20px;'> Deaths: " + str(l[1]) + "</span>"+
             "<span style='color: lightgreen; font-size:30px; margin-left:20px;'> Recovered: " + str(l[2]) + "</span>"+
             "</div>")
               )
cou=list(confirmed_df.country)
cou.insert(0,'World')
cou=list(dict.fromkeys(cou))
interact(country_data, country=cou);


# Slide to compare data of Countries:

In [ ]:
def highlight_col(x):
    y = 'background-color: #8B6914'
    r = 'background-color: maroon'
    g = 'background-color: darkgreen'
    temp_df = pd.DataFrame('', index=x.index, columns = x.columns)
    temp_df.iloc[:,1]=y
    temp_df.iloc[:,2]=r
    temp_df.iloc[:,3]=g
    return temp_df
d=sorted_country_df[['country','confirmed','deaths','recovered']]
d.index=[x+1 for x in range(len(d.values))]
def get_highlight_col(N):
    return d.head(N).style.apply(highlight_col, axis=None)
interact(get_highlight_col, N=(1,30));

In [ ]:
import plotly.express as px

# Slide to compare worst hit Countries:

In [ ]:
def worst_hit(Number):
    fig = px.scatter(sorted_country_df.head(Number), x='country', y='confirmed', size='confirmed', color='country', 
                     hover_name='country', size_max=60)
    fig.update_layout(
        {'plot_bgcolor':'#8EE5EE',
        'paper_bgcolor':'#838B83',
        },
        title=" Worst hit countries",
        xaxis_title="Dates",
        yaxis_title="Confirmed and Death Cases",
        font=dict(
            family="TimesNewRoman",
            size=14,
            color='#F8F8FF'
        ),           
        width = 800
    )
    fig.show()
#worst_hit(10)
interact(worst_hit, Number=(1,30));


# Select the Country to get data:

In [ ]:
def plot_cases_of_a_country(country):
    labels = ['confirmed', 'deaths']
    colors = ['blue', 'red']
    mode_size = [6, 8]
    line_size = [4, 5]
    
    df_list = [confirmed_df, death_df]
    
    fig = go.Figure();
    
    for i, df in enumerate(df_list):
        if country == 'World' or country == 'world':
            x_data = np.array(list(df.iloc[:, 20:].columns))
            y_data = np.sum(np.asarray(df.iloc[:,4:]),axis = 0)
            
        else:    
            x_data = np.array(list(df.iloc[:, 20:].columns))
            y_data = np.sum(np.asarray(df[df['country'] == country].iloc[:,20:]),axis = 0)
            
        fig.add_trace(go.Scatter(x=x_data, y=y_data, mode='lines+markers',
        name=labels[i],
        line=dict(color=colors[i], width=line_size[i]),
        connectgaps=True,
        text = "Total " + str(labels[i]) +": "+ str(y_data[-1])
        ));
        
    fig.update_layout({'plot_bgcolor':'#8EE5EE',
        'paper_bgcolor':'#838B83',
        },
        title="Confirmed and Death cases",
        xaxis_title="Dates",
        yaxis_title="Confirmed and Death Cases",
        font=dict(
            family="TimesNewRoman",
            size=14,
            color='#F8F8FF'
        ),           
        width = 800         )
    fig.show();
#plot_cases_of_a_country('India') 
cou=list(confirmed_df.country)
cou.insert(0,'World')
cou=list(dict.fromkeys(cou))
interact(plot_cases_of_a_country, country=cou);
    
    

***
# <font color='yellow'>Top 10 worst affected Countries- Confirmed cases</font>
***

In [ ]:
fig1=px.bar(
    sorted_country_df.head(10),
    x = "country",
    y = "confirmed",
    #title= "Top 10 worst affected countries- Confirmed cases", # the axis names
    color_discrete_sequence=["yellow"], 
    height=500,
    width=800
)
fig1.update_layout({'plot_bgcolor':'#8EE5EE',
        'paper_bgcolor':'#838B83',
        },
        title=" Compared using Confirmed cases",
        xaxis_title="Countries",
        yaxis_title="Confirmed Cases",
        font=dict(
            family="TimesNewRoman",
            size=14,
            color='#F8F8FF'
        ),           
        width = 800         )
fig1.show()

***
# <font color='red'>Top 10 worst affected Countries- Death/Confirmed ratio</font>
***

In [ ]:
country_df['d/c']=country_df['deaths']/country_df['confirmed']
d= country_df.sort_values('d/c', ascending=False)
#sorted_death_df = country_df.sort_values(country_df['d/c'], ascending=False)
fig2=px.bar(
    d.head(10),
    x = "country",
    y = "deaths",
    #title= "Top 10 worst affected countries- Death cases", # the axis names
    color_discrete_sequence=["red"], 
    height=500,
    width=800
)
fig2.update_layout(
        {'plot_bgcolor':'#8EE5EE',
        'paper_bgcolor':'#838B83',
        },
        title=" Compared using Death/Confirmed cases",
        xaxis_title="Countries",
        yaxis_title="Confirmed Cases",
        font=dict(
            family="TimesNewRoman",
            size=14,
            color='#F8F8FF'
        ),           
        width = 800
    )
fig2.show()


***
# <font color='green'>Top 10 worst affected Countries-Recovered/Confirmed ratio</font>
***

In [ ]:
country_df['r/c']=country_df['recovered']/country_df['confirmed']
r= country_df.sort_values('r/c', ascending=False)
#sorted_recovered_df = country_df.sort_values('recovered', ascending=True)
fig3=px.bar(
    r.head(10),
    x = "country",
    y = "recovered",
    #title= "Top 10 worst affected countries-Recovered cases", # the axis names
    color_discrete_sequence=["green"], 
    height=500,
    width=800
)
fig3.update_layout(
        {'plot_bgcolor':'#8EE5EE',
        'paper_bgcolor':'#838B83',
        },
        title=" Compared using Recovered/Confirmed cases",
        xaxis_title="Countries",
        yaxis_title="Confirmed Cases",
        font=dict(
            family="TimesNewRoman",
            size=14,
            color='#F8F8FF'
        ),           
        width = 800
    )
fig3.show()

# Select your State to get data:

In [ ]:
list_date=new_data.loc[new_data['Country_Region'] == 'India',['Updated']].tail(3)
list_date=[x for x in list_date.iloc[:,0]]


In [ ]:
def state_data3(State='Uttar Pradesh'):
    data_state=new_data[(new_data['AdminRegion1'] == State)&(new_data['AdminRegion2'].isnull())&((new_data['Updated'] == list_date[2])|(new_data['Updated'] == list_date[1])|(new_data['Updated'] == list_date[0]))].tail(1)
    data_state.fillna(0, inplace=True)
    if data_state.empty:
        print('No data found')
    else:
        display(HTML("<div style = 'background-color: #504e4e; padding: 30px '>" +
             "<span style='color: yellow; font-size:30px;'> Confirmed: "  + str(int(data_state.iloc[0,2])) +"</span>" +
             "<span style='color: red; font-size:30px;margin-left:20px;'> Deaths: " + str(int(data_state.iloc[0,4])) + "</span>"+
             "<span style='color: lightgreen; font-size:30px; margin-left:20px;'> Recovered: " + str(int(data_state.iloc[0,6])) + "</span>"+
             "</div>")
               )
list_state=list(new_data.AdminRegion1)
list_state=[x for x in list_state if str(x)!='nan']
#print(list_state)
list_state=list(dict.fromkeys(list_state))
list_state.sort()
#print(list_state)
#list_state=list_state[1:]
interact(state_data3, State=list_state);

# Select your City to get data:

#### Use first letter capital

In [ ]:
def city_data3(city='Lucknow'):
    data_city=new_data[(new_data['AdminRegion2'] == city)&((new_data['Updated'] == list_date[2])|(new_data['Updated'] == list_date[1])|(new_data['Updated'] == list_date[0]))]
    data_city=data_city.tail(1)
    data_city.fillna(0, inplace=True)
    if data_city.empty:
        print('No data found')
    else:
        display(HTML("<div style = 'background-color: #504e4e; padding: 30px '>" +
             "<span style='color: yellow; font-size:30px;'> Confirmed: "  + str(int(data_city.iloc[0,2])) +"</span>" +
             "<span style='color: red; font-size:30px;margin-left:20px;'> Deaths: " + str(int(data_city.iloc[0,4])) + "</span>"+
             "<span style='color: lightgreen; font-size:30px; margin-left:20px;'> Recovered: " + str(int(data_city.iloc[0,6])) + "</span>"+
             "</div>")
               )

interact(city_data3, city='Lucknow');    

### If you liked my work, please press the like button below: